In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
import cartopy.feature as cfeature
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d 
from scipy import ndimage
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.models import load_model
from sklearn.cluster import KMeans, MiniBatchKMeans
import joblib
import plot_functions
import importlib 
importlib.reload(plot_functions)
from plot_functions import *
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from autoencoder import SobelFilterLayer, SimpleAutoencoder


total cores: 256


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import socket
hostname = socket.gethostname()
if "nird" in hostname:
    tf.config.threading.set_inter_op_parallelism_threads(8)
    data_loc = "/nird/projects/NS9600K/fslippe/data/"
    folder = "/nird/projects/NS9600K/data/modis/cao/MOD02/2020/ /nird/projects/NS9600K/data/modis/cao/MOD02/2021/ /nird/projects/NS9600K/data/modis/cao/MOD02/2023/"
if "mimi" in hostname:
    data_loc = "/uio/hume/student-u37/fslippe/data/"
    folder = "/scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/ /scratch/fslippe/modis/MOD02/cao_test_data/"


bands=[29]
max_vals = np.load("%smodels/max_vals_dnb_l95_z50_ps128_(29)_cao_months_202012-202111.npy" %data_loc)
print(len(bands))

#encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")
encoder = load_model("%smodels/encoder_scheduler_250k_dnb_l90_z50_fcao_months_202012_(29)_202111-64" %data_loc)

1 Physical GPUs, 1 Logical GPUs


2023-12-13 18:42:26.827180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20424 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


1


In [3]:
### EXTRACT CAO AND NOn CAO CASES
import importlib
import extract_training_data
importlib.reload(extract_training_data)
from extract_training_data import *
patch_size = 128

start = "20230101"
end = "20230430"
dates = generate_date_list(start, end)
start = "20231001"
end = "20231231"
dates.extend(generate_date_list(start, end))

bands = [29]
folder = "/scratch/fslippe/modis/MOD02/2019/ /scratch/fslippe/modis/MOD02/2020/ /scratch/fslippe/modis/MOD02/2021/ /scratch/fslippe/modis/MOD02/2022/ /scratch/fslippe/modis/MOD02/2023/"
x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao = extract_1km_data(folder,
                                                         bands=bands,
                                                         date_list=dates,
                                                         return_lon_lat=True,
                                                         data_loc=data_loc,
                                                         data_type="npy",
                                                         combine_pics=True)

x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao = zip(*[(xi, date, mask, lon_lat, mod_min) for xi, date, mask, lon_lat, mod_min in zip(x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao) if (xi.shape[0] > patch_size) and (xi.shape[1] > patch_size)])
x_cao = list(x_cao)
dates_cao = list(dates_cao)
len(x_cao)

total cores: 256
['/scratch/fslippe/modis/MOD02/2019/', '/scratch/fslippe/modis/MOD02/2020/', '/scratch/fslippe/modis/MOD02/2021/', '/scratch/fslippe/modis/MOD02/2022/', '/scratch/fslippe/modis/MOD02/2023/']
['2019001', '2019002', '2019003', '2019004', '2019005', '2019006', '2019007', '2019008', '2019009', '2019010', '2019011', '2019012', '2019013', '2019014', '2019015', '2019016', '2019017', '2019018', '2019019', '2019020', '2019021', '2019022', '2019023', '2019024', '2019025', '2019026', '2019027', '2019028', '2019029', '2019030', '2019031', '2019032', '2019033', '2019034', '2019035', '2019036', '2019037', '2019038', '2019039', '2019040', '2019041', '2019042', '2019043', '2019044', '2019045', '2019046', '2019047', '2019048', '2019049', '2019050', '2019051', '2019052', '2019053', '2019054', '2019055', '2019056', '2019057', '2019058', '2019059', '2019060', '2019061', '2019062', '2019063', '2019064', '2019065', '2019066', '2019067', '2019068', '2019069', '2019070', '2019071', '2019072',

100%|██████████| 120/120 [01:43<00:00,  1.16it/s]


1438
1438


1229

In [4]:
gc.collect()
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)

patches, all_lon_patches, all_lat_patches, starts, ends, shapes, n_patches_tot, indices = generate_patches([xi[:,:,0] for xi in x_cao],
                                                                                                            masks_cao,
                                                                                                            lon_lats_cao,
                                                                                                            max_vals,
                                                                                                            autoencoder_predict,
                                                                                                            strides=[1, patch_size, patch_size,1])
n_patches = len(patches)

In [5]:
cluster = joblib.load("/uio/hume/student-u37/fslippe/data/models/cluster_dnb_l90_z50_ps128_f64_(29)_%s-%s.pkl" %("cao_months_202012", "202111"))

n_chunks = 10
chunk_size = n_patches // n_chunks

patch_chunks = [patches[i * chunk_size : (i + 1) * chunk_size] for i in range(n_chunks)]

# Handle the remaining patches for the case where n_patches is not exactly divisible by n_chunks
if n_patches % n_chunks != 0:
    patch_chunks.append(patches[n_chunks * chunk_size:])

encoded_patch_chunks = [encoder.predict(chunk) for chunk in patch_chunks]
encoded_patches = np.concatenate(encoded_patch_chunks)

encoded_patches_flat = encoded_patches.reshape(encoded_patches.shape[0], -1)
gc.collect()


2023-12-13 18:49:59.629870: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


10222

In [6]:
encoder = load_model("/uio/hume/student-u37/fslippe/data/models/encoder_scheduler_250k_dnb_l90_z50_fcao_months_202012_(29)_202111-64")
labels = cluster.predict(encoded_patches_flat)

In [7]:
global_max = np.max([np.max(cm) for cm in cluster.labels_])+1
projection = ccrs.Stereographic(central_latitude=90)
target_months = [1, 2, 3, 4, 10, 11, 12]
import functions
importlib.reload(functions)
from functions import * 
size_thresholds = [3,4,5,6,7]
desired_label = [3, 0]
grid_resolution = 128e3
filters = 64
year = dates_cao[0][:4]
for size_threshold in size_thresholds:
    for target_month in target_months:
        print("/uio/hume/student-u37/fslippe/data/hist_maps/%s/hist_counts_res%s_thr%s_ps%s_filters%s_%s%s" %(year, int(grid_resolution*1e-3), size_threshold, patch_size, filters, year, str(target_month).zfill(2)))
        idx = np.array([i for i, date_str in enumerate(dates_cao) if datetime.datetime.strptime(date_str, '%Y%j').month == target_month])
        x_grid, y_grid, counts = generate_hist_map(
                                    [n_patches_tot[i] for i in idx],
                                    [indices[i] for i in idx],
                                    labels,
                                    [starts[i] for i in idx],
                                    [ends[i] for i in idx],  
                                    [shapes[i] for i in idx],
                                    [all_lon_patches[i] for i in idx],
                                    [all_lat_patches[i] for i in idx],
                                    [dates_cao[i] for i in idx],
                                    desired_label=desired_label,
                                    size_threshold=size_threshold,
                                    patch_size=patch_size,
                                    global_max=global_max,
                                    projection=projection,
                                    grid_resolution=grid_resolution)
        tot_days = len(set(np.array(dates_cao)[idx]))
        ds = {"days": tot_days, "counts": counts}
        np.save("/uio/hume/student-u37/fslippe/data/hist_maps/%s/hist_counts_res%s_thr%s_ps%s_filters%s_%s%s" %(year, int(grid_resolution*1e-3), size_threshold, patch_size, filters, year, str(target_month).zfill(2)), ds)



/uio/hume/student-u37/fslippe/data/hist_maps/2023/hist_counts_res128_thr3_ps128_filters64_202301
/uio/hume/student-u37/fslippe/data/hist_maps/2023/hist_counts_res128_thr3_ps128_filters64_202302
/uio/hume/student-u37/fslippe/data/hist_maps/2023/hist_counts_res128_thr3_ps128_filters64_202303
/uio/hume/student-u37/fslippe/data/hist_maps/2023/hist_counts_res128_thr3_ps128_filters64_202304
/uio/hume/student-u37/fslippe/data/hist_maps/2023/hist_counts_res128_thr3_ps128_filters64_202310


IndexError: arrays used as indices must be of integer (or boolean) type

In [10]:
ds = np.load("/uio/hume/student-u37/fslippe/data/hist_maps/%s/hist_counts_res%s_thr%s_ps%s_filters%s_%s%s.npy" %(year, int(grid_resolution*1e-3), 6, patch_size, filters, year, str(1).zfill(2)), allow_pickle=True).item()
counts = ds["counts"]
tot_days = ds["days"]

plot_hist_map(x_grid, y_grid, counts, tot_days, projection, title="Percentage of time with predicted CAO", extent=[-36, 36, 58.2, 82], levels=10, cmap="turbo")


FileNotFoundError: [Errno 2] No such file or directory: '/uio/hume/student-u37/fslippe/data/hist_maps/2023/hist_counts_res128_thr6_ps128_filters64_202301.npy'

In [ ]:
# Visualize the result
import plot_functions
importlib.reload(plot_functions)
from plot_functions import * 
tot_days = len(set(np.array(dates_cao)[idx]))
print(tot_days)
plot_hist_map(x_grid, y_grid, counts, tot_days, projection, title="Percentage of time with predicted CAO", extent=[-36, 36, 58.2, 82], levels=10, cmap="turbo")


31


KeyboardInterrupt: 